In [1]:
# at the very top of your notebook:
%load_ext autoreload
%autoreload 2


In [ ]:
import ast
import math
import random
import json
import warnings
from collections import Counter, deque
import copy
import os
import re

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import (
    accuracy_score, average_precision_score, f1_score,
    precision_score, recall_score, roc_auc_score, classification_report,
    precision_recall_curve
)
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler

from iterstrat.ml_stratifiers import (
    MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit
)
from pathlib import Path

from utils_corrected import (
    compute_ensemble_probs, predict_without_MCD,
    chunked_mad_over_runs, build_pred_annots_dict, load_kfold_ensembles, PFP
)

In [2]:
#read in TUNED_MODEL_ARCHS.json
with open('TUNED_MODEL_ARCHS.json') as f:
    TUNED_MODEL_ARCHS = json.load(f)

Load models

In [9]:
MF_CHOSEN_CONFIG = TUNED_MODEL_ARCHS['MFO']['CHOSEN_CONFIG']
MF_CHOSEN_CONFIG_ARCH = TUNED_MODEL_ARCHS['MFO']['CHOSEN_CONFIG_ARCH']
MF_CHOSEN_CONFIG_LOSS = 'Balanced'
MF_CHOSEN_LEARNING_RATE = TUNED_MODEL_ARCHS['MFO']['CHOSEN_CONFIG_LR']
MF_CHOSEN_NUM_CLASSES = TUNED_MODEL_ARCHS['MFO']['NUM_CLASSES']
MF_dropout_rate   = 0.3
MF_hidden_dims    = MF_CHOSEN_CONFIG_ARCH
MF_input_dim      = 512
MF_output_dim     = MF_CHOSEN_NUM_CLASSES
MF_device         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# MF_base_kwargs should match what PFP expects, e.g. {"input_dim":..., "hidden_dims":..., ...}
MF_base_kwargs    = {
    'input_dim'   : MF_input_dim,
    'hidden_dims' : MF_hidden_dims,
    'output_dim'  : MF_output_dim,
    'dropout_rate': MF_dropout_rate,
}

BP_CHOSEN_CONFIG = TUNED_MODEL_ARCHS['BPO']['CHOSEN_CONFIG']
BP_CHOSEN_CONFIG_ARCH = TUNED_MODEL_ARCHS['BPO']['CHOSEN_CONFIG_ARCH']
BP_CHOSEN_CONFIG_LOSS = 'Balanced'
BP_CHOSEN_LEARNING_RATE = TUNED_MODEL_ARCHS['BPO']['CHOSEN_CONFIG_LR']
BP_CHOSEN_NUM_CLASSES = TUNED_MODEL_ARCHS['BPO']['NUM_CLASSES']
BP_dropout_rate   = 0.3
BP_hidden_dims    = BP_CHOSEN_CONFIG_ARCH
BP_input_dim      = 512
BP_output_dim     = BP_CHOSEN_NUM_CLASSES
BP_device         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# BP_base_kwargs should match what PFP expects, e.g. {"input_dim":..., "hidden_dims":..., ...}
BP_base_kwargs    = {
    'input_dim'   : BP_input_dim,
    'hidden_dims' : BP_hidden_dims,
    'output_dim'  : BP_output_dim,
    'dropout_rate': BP_dropout_rate,
}

CC_CHOSEN_CONFIG = TUNED_MODEL_ARCHS['CCO']['CHOSEN_CONFIG']
CC_CHOSEN_CONFIG_ARCH = TUNED_MODEL_ARCHS['CCO']['CHOSEN_CONFIG_ARCH']
CC_CHOSEN_CONFIG_LOSS = 'Balanced'
CC_CHOSEN_LEARNING_RATE = TUNED_MODEL_ARCHS['CCO']['CHOSEN_CONFIG_LR']
CC_CHOSEN_NUM_CLASSES = TUNED_MODEL_ARCHS['CCO']['NUM_CLASSES']
CC_dropout_rate   = 0.3
CC_hidden_dims    = CC_CHOSEN_CONFIG_ARCH
CC_input_dim      = 512
CC_output_dim     = CC_CHOSEN_NUM_CLASSES
CC_device         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# CC_base_kwargs should match what PFP expects, e.g. {"input_dim":..., "hidden_dims":..., ...}
CC_base_kwargs    = {
    'input_dim'   : CC_input_dim,
    'hidden_dims' : CC_hidden_dims,
    'output_dim'  : CC_output_dim,
    'dropout_rate': CC_dropout_rate,
}

MF_root_dir     = r"mf_publish"
BP_root_dir     = r"bp_publish"
CC_root_dir     = r"cc_publish"

MF_kf_root      = os.path.join(MF_root_dir, "k_folds")
BP_kf_root      = os.path.join(BP_root_dir, "k_folds")
CC_kf_root      = os.path.join(CC_root_dir, "k_folds")

MF_fold_options = [20]
BP_fold_options = [20]
CC_fold_options = [20]

MF_raw_ensembles = load_kfold_ensembles(
    kf_root=MF_kf_root,
    fold_options=MF_fold_options,
    model_cls=PFP,
    model_kwargs=MF_base_kwargs,
    device=MF_device
)
BP_raw_ensembles = load_kfold_ensembles(
    kf_root=BP_kf_root,
    fold_options=BP_fold_options,
    model_cls=PFP,
    model_kwargs=BP_base_kwargs,
    device=BP_device
)
CC_raw_ensembles = load_kfold_ensembles(
    kf_root=CC_kf_root,
    fold_options=CC_fold_options,
    model_cls=PFP,
    model_kwargs=CC_base_kwargs,
    device=CC_device
)

Identify which genes are missing annotations in each subontology

In [36]:
# === CONFIG ===
dict_in_dir = Path(r"genomes_to_annotate_with_PlasmoFP/gene_dicts_out_complete_and_filtered_2")
out_dir     = Path(r"genomes_to_annotate_with_PlasmoFP/missing_by_aspect_new_2")
out_dir.mkdir(exist_ok=True)

# which GO keys correspond to each aspect
aspect_keys = {
    "MF": ["GO Function", "GO IEA Function"],
    "BP": ["GO Process",  "GO IEA Process"],
    "CC": ["GO Component","GO IEA Component"]
}

for pkl_fp in sorted(dict_in_dir.glob("*_gene_dict_complete.pkl")):
    species = pkl_fp.stem.rsplit("_gene_dict", 1)[0]

    # load the full gene_dict
    with open(pkl_fp, "rb") as f:
        full_dict = pickle.load(f)

    for asp, keys in aspect_keys.items():
        missing_dict   = {}
        missing_embeds = []

        for gene_id, rec in full_dict.items():
            emb = rec["embedding"]
            # skip genes without any embedding
            if emb is None:
                continue

            # sum of both buckets for this aspect
            has_terms = sum(len(rec[k]) for k in keys)
            if has_terms == 0:
                # this gene has NO terms in this aspect; include it
                missing_dict[gene_id] = rec
                missing_embeds.append(emb)

        # stack into array (n_missing × emb_dim)
        if missing_embeds:
            emb_arr = np.stack(missing_embeds, axis=0)
        else:
            # grab emb_dim from any entry
            sample = next(iter(full_dict.values()))
            emb_dim = sample["embedding"].shape[0] if sample["embedding"] is not None else 0
            emb_arr = np.zeros((0, emb_dim))

        # save subset dict
        out_pkl = out_dir / f"{species}_missing_{asp}_gene_dict.pkl"
        with open(out_pkl, "wb") as f:
            pickle.dump(missing_dict, f)

        # save embeddings
        out_npy = out_dir / f"{species}_missing_{asp}_embeddings.npy"
        np.save(out_npy, emb_arr)

        print(f"{species} [{asp}]: {len(missing_dict)} genes missing (with embeddings) → "
              f"{out_pkl.name}, {out_npy.name}")


PlasmoDB-68_PadleriG01 [MF]: 2422 genes missing (with embeddings) → PlasmoDB-68_PadleriG01_missing_MF_gene_dict.pkl, PlasmoDB-68_PadleriG01_missing_MF_embeddings.npy
PlasmoDB-68_PadleriG01 [BP]: 2863 genes missing (with embeddings) → PlasmoDB-68_PadleriG01_missing_BP_gene_dict.pkl, PlasmoDB-68_PadleriG01_missing_BP_embeddings.npy
PlasmoDB-68_PadleriG01 [CC]: 2453 genes missing (with embeddings) → PlasmoDB-68_PadleriG01_missing_CC_gene_dict.pkl, PlasmoDB-68_PadleriG01_missing_CC_embeddings.npy
PlasmoDB-68_PbergheiANKA [MF]: 1992 genes missing (with embeddings) → PlasmoDB-68_PbergheiANKA_missing_MF_gene_dict.pkl, PlasmoDB-68_PbergheiANKA_missing_MF_embeddings.npy
PlasmoDB-68_PbergheiANKA [BP]: 2272 genes missing (with embeddings) → PlasmoDB-68_PbergheiANKA_missing_BP_gene_dict.pkl, PlasmoDB-68_PbergheiANKA_missing_BP_embeddings.npy
PlasmoDB-68_PbergheiANKA [CC]: 1712 genes missing (with embeddings) → PlasmoDB-68_PbergheiANKA_missing_CC_gene_dict.pkl, PlasmoDB-68_PbergheiANKA_missing_CC_e

Use PlasmoFP to predict missing subontologies 

In [37]:
missing_dir = Path(r"genomes_to_annotate_with_PlasmoFP/missing_by_aspect_new_2")
output_dir = missing_dir / "ensemble_probs"
output_dir.mkdir(exist_ok=True)

aspect_to_ensembles = {
    "MF": MF_raw_ensembles,
    "BP": BP_raw_ensembles,
    "CC": CC_raw_ensembles,
}

batch_size = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pattern = re.compile(r"(?P<species>.+)_missing_(?P<asp>MF|BP|CC)_embeddings\.npy")

files = sorted(missing_dir.glob("*_missing_*_embeddings.npy"))
for npy_fp in tqdm(files, desc="Processing embeddings"):
    m = pattern.match(npy_fp.name)
    if not m:
        continue
    species = m.group("species")
    asp = m.group("asp")

    # Load embeddings and create DataLoader
    emb_arr = np.load(npy_fp)
    loader = DataLoader(
        TensorDataset(torch.from_numpy(emb_arr).float()),
        batch_size=batch_size,
        shuffle=False
    )

    
    kf_models = aspect_to_ensembles[asp]
    
    
    probs_dict = compute_ensemble_probs(kf_models, loader, device, predict_without_MCD)
    
    
    probs_20fold = probs_dict[20]  # Shape: (20, N, C)
    
    
    median_probs = np.median(probs_20fold, axis=0)  # Shape: (N, C)
    mad_probs = chunked_mad_over_runs(probs_20fold, chunk_size=50)  # Shape: (N, C)
    
    
    median_fp = output_dir / f"{species}_{asp}_median_20fold.npy"
    mad_fp = output_dir / f"{species}_{asp}_mad_20fold.npy"
    np.save(median_fp, median_probs)
    np.save(mad_fp, mad_probs)
    
    tqdm.write(f"{species} [{asp}]: median={median_probs.shape}, mad={mad_probs.shape}")

print("Median/MAD files saved")


Processing embeddings:   2%|▏         | 1/57 [00:06<05:42,  6.12s/it]

PlasmoDB-68_PadleriG01 [BP]: median=(2863, 1548), mad=(2863, 1548)


Processing embeddings:   4%|▎         | 2/57 [00:07<03:02,  3.31s/it]

PlasmoDB-68_PadleriG01 [CC]: median=(2453, 255), mad=(2453, 255)


Processing embeddings:   5%|▌         | 3/57 [00:10<02:45,  3.06s/it]

...

PlasmoDB-68_Pyoeliiyoelii17XNL2023 [BP]: median=(2564, 1548), mad=(2564, 1548)


Processing embeddings:  98%|█████████▊| 56/57 [03:05<00:02,  2.96s/it]

PlasmoDB-68_Pyoeliiyoelii17XNL2023 [CC]: median=(2961, 255), mad=(2961, 255)


Processing embeddings: 100%|██████████| 57/57 [03:07<00:00,  3.30s/it]

PlasmoDB-68_Pyoeliiyoelii17XNL2023 [MF]: median=(2169, 744), mad=(2169, 744)
Median/MAD files saved


In [38]:
output_dir

PosixPath('genomes_to_annotate_with_PlasmoFP/missing_by_aspect_new_2/ensemble_probs')

In [47]:
eff_thresh = {
    "Function": json.load(open("effective_thresholds_ensemble20_function.json")),
    "Process": json.load(open("effective_thresholds_ensemble20_process.json")),
    "Component": json.load(open("effective_thresholds_ensemble20_component.json")),
}

for aspect, term_dict in eff_thresh.items():
    for term, fdr_map in term_dict.items():
        new_map = {}
        for fdr_str, thr in fdr_map.items():
            if thr is not None:
                new_map[round(float(fdr_str), 2)] = thr
        eff_thresh[aspect][term] = new_map

mlb_by_asp = {
    "MF": pickle.load(open("function_mlb.pkl", "rb")),
    "BP": pickle.load(open("process_mlb.pkl", "rb")),
    "CC": pickle.load(open("component_mlb.pkl", "rb"))
}

median_files = sorted(output_dir.glob("*_median_20fold.npy"))
all_predictions = {}

for med_fp in tqdm(median_files, desc="Generating predictions with corrected thresholds", leave=False):
    parts = med_fp.stem.split("_")
    asp = parts[-3]  # Extract aspect (MF/BP/CC)
    species = "_".join(parts[:-3])  # Extract species name
    
    median_probs = np.load(med_fp)
    mad_fp = med_fp.with_name(med_fp.name.replace("_median_", "_mad_"))
    MAD_probs = np.load(mad_fp)
    
    gene_dict_fp = missing_dir / f"{species}_missing_{asp}_gene_dict.pkl"
    gene_dict = pickle.load(open(gene_dict_fp, "rb"))
    test_entries = list(gene_dict.keys())
    
    aspect_name = {"MF": "Function", "BP": "Process", "CC": "Component"}[asp]
    thresholds = eff_thresh[aspect_name]
    mlb = mlb_by_asp[asp]
    
    preds_for_aspect = {}
    target_fdrs = [0.01, 0.05, 0.10, 0.20, 0.30]
    
    for target_fdr in target_fdrs:
        tqdm.write(f"{species}/{asp} {target_fdr}")
        pred_dict = build_pred_annots_dict(
            median_probs, MAD_probs, thresholds,
            test_entries=test_entries,
            terms=list(mlb.classes_),
            lambda_val=1,
            target_fdr=target_fdr
        )
        preds_for_aspect[round(target_fdr, 3)] = pred_dict
    
    all_predictions.setdefault(species, {})[asp] = preds_for_aspect

out_path = missing_dir / "all_PUF_predictions.pkl"
with open(out_path, "wb") as f:
    pickle.dump(all_predictions, f)

/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
Generating predictions with corrected thresholds:   0%|          | 0/57 [00:00<?, ?it/s]

PlasmoDB-68_PadleriG01/BP 0.01


Building predictions: 100%|██████████| 2863/2863 [00:39<00:00, 72.14it/s]
Generating predictions with corrected thresholds:   0%|          | 0/57 [00:39<?, ?it/s]

PlasmoDB-68_PadleriG01/BP 0.05


Building predictions: 100%|██████████| 2863/2863 [00:38<00:00, 73.94it/s]
Generating predictions with corrected thresholds:   0%|          | 0/57 [01:18<?, ?it/s]

...

Building predictions: 100%|██████████| 2169/2169 [00:13<00:00, 166.26it/s]
Generating predictions with corrected thresholds:  98%|█████████▊| 56/57 [1:31:40<01:22, 82.58s/it]

PlasmoDB-68_Pyoeliiyoelii17XNL2023/MF 0.2


Building predictions: 100%|██████████| 2169/2169 [00:13<00:00, 166.71it/s]
Generating predictions with corrected thresholds:  98%|█████████▊| 56/57 [1:31:53<01:22, 82.58s/it]

PlasmoDB-68_Pyoeliiyoelii17XNL2023/MF 0.3


Building predictions: 100%|██████████| 2169/2169 [00:13<00:00, 166.06it/s]


Select deepest terms after prediction

In [ ]:
with open("raw_data_from_uniprot/function_ALL_data_ancestors_dict.json") as f:
    function_ancestors_dict = json.load(f)
with open("raw_data_from_uniprot/function_ALL_data_json_dict.json") as f:
    function_json_dict = json.load(f)
with open("raw_data_from_uniprot/process_ALL_data_ancestors_dict.json") as f:
    process_ancestors_dict = json.load(f)
with open("raw_data_from_uniprot/process_ALL_data_json_dict.json") as f:
    process_json_dict = json.load(f)
with open("raw_data_from_uniprot/component_ALL_data_ancestors_dict.json") as f:
    component_ancestors_dict = json.load(f)
with open("raw_data_from_uniprot/component_ALL_data_json_dict.json") as f:
    component_json_dict = json.load(f)

In [49]:
def select_deepest_terms(pred_dict, ancestors_dict, filtered_json_dict, mlb):
    """
    For each protein in pred_dict (mapping protein to {GO term: effective score}),
    select only the deepest GO terms that have all their ancestors present.
    
    Returns:
      dict: Mapping from protein entry -> dict of {GO term: effective score} for the deepest predictions.
    """
    deepest_predictions = {}
    
    for entry, go_dict in pred_dict.items():
        # Start with the set of predicted terms that are also in the MLb classes.
        predicted_terms = {term: score for term, score in go_dict.items() if term in mlb.classes_}
        
        # Step 1: Ensure ancestor consistency: only keep terms where, if defined,
        # all their ancestors are also predicted.
        valid_terms = {}
        for term, score in predicted_terms.items():
            if term in ancestors_dict:
                if set(ancestors_dict[term]).issubset(predicted_terms.keys()):
                    valid_terms[term] = score
            else:
                valid_terms[term] = score
        
        # Step 2: Select the deepest terms.
        # Remove any term that has a child in the valid set.
        deepest_terms = dict(valid_terms)  
        for term in valid_terms:
            if term in filtered_json_dict:
                children = set(filtered_json_dict[term])
                # If any child of term is also in valid_terms, remove the parent term.
                if children & valid_terms.keys():
                    deepest_terms.pop(term, None)
        
        deepest_predictions[entry] = deepest_terms  # mapping term -> effective score
    
    return deepest_predictions

def annotate_go_terms(predictions_dict, json_dict):
    """
    Convert a predictions dictionary mapping each protein to a mapping of
    GO term to effective score into a dictionary mapping each protein to a set
    of (GO term, effective score, GO term name) tuples.
    """
    annotated_dict = {}
    for protein, term_dict in predictions_dict.items():
        annotated_dict[protein] = {
            (term, score, json_dict.get(term, {}).get('name', term))
            for term, score in term_dict.items()
        }
    return annotated_dict

In [50]:
anc_dicts = {
    "MF": function_ancestors_dict,
    "BP": process_ancestors_dict,
    "CC": component_ancestors_dict,
}

json_dicts = {
    "MF": function_json_dict,
    "BP": process_json_dict,
    "CC": component_json_dict,
}

filtered_json_dict = {}
ancestors_dict_filtered = {}

for asp in ["MF", "BP", "CC"]:
    mlb   = mlb_by_asp[asp]
    anc   = anc_dicts[asp]
    js    = json_dicts[asp]

    # 1) Filter JSON children to only those in mlb.classes_
    fj = {}
    for term, info in js.items():
        children = info.get("children", [])
        fj[term] = [c["id"] for c in children if c["id"] in mlb.classes_]
    filtered_json_dict[asp] = fj

    # 2) Filter ancestor lists to only include mlb.classes_
    fa = {}
    for term, parents in anc.items():
        if parents:
            valid = [p for p in parents if p in mlb.classes_]
            if valid:
                fa[term] = valid
    ancestors_dict_filtered[asp] = fa

In [51]:
target_fdrs = [0.01, 0.05, 0.10, 0.20, 0.30]

deepest_annotations = {}

for species, asp_dict in all_predictions.items():
    deepest_annotations[species] = {}
    for asp, preds_for_aspect in asp_dict.items():
        anc_dict = ancestors_dict_filtered[asp]
        js_dict  = json_dicts[asp]
        child_map = filtered_json_dict[asp]

        deepest_annotations[species][asp] = {}

        for fdr in tqdm(target_fdrs, desc=f"{species}/{asp}", leave=False):
            if fdr not in preds_for_aspect:
                continue  # skip if this FDR wasn't generated

            pred_dict = preds_for_aspect[fdr]

            # select only the deepest terms
            deepest = select_deepest_terms(
                pred_dict,
                anc_dict,
                child_map,
                mlb_by_asp[asp]
            )
            # annotate them
            annots = annotate_go_terms(deepest, js_dict)
            deepest_annotations[species][asp][fdr] = annots

Get the depths for the deepest terms 

In [52]:
from goatools.obo_parser import GODag
import numpy as np

obodag = GODag("go-basic.obo")

# 1) Build deepest_levels from deepest_annotations
deepest_levels = {}

for species, asp_dict in deepest_annotations.items():
    deepest_levels[species] = {}
    for asp, fdr_dict in asp_dict.items():
        deepest_levels[species][asp] = {}
        for fdr, annots in fdr_dict.items():
            # annots: {protein: {(go_id, score, name), …}, …}
            level_map = {}
            for prot, terms in annots.items():
                lvl_set = set()
                for go_id, score, name in terms:
                    depth = obodag[go_id].depth if go_id in obodag else np.nan
                    lvl_set.add((go_id, name, depth))
                level_map[prot] = lvl_set
            deepest_levels[species][asp][fdr] = level_map


go-basic.obo: fmt(1.2) rel(2024-09-08) 44,296 Terms


In [53]:
import pandas as pd
from collections import Counter
from pathlib import Path

out_dir = Path("genomes_to_annotate_with_PlasmoFP/depth_counts_for_deepest_terms_2")
out_dir.mkdir(exist_ok=True)

target_fdrs = [0.01, 0.05, 0.10, 0.20, 0.30]

for species, asp_dict in deepest_levels.items():
    for asp, fdr_dict in asp_dict.items():
        depth_counters = {}
        for fdr in target_fdrs:
            lvl_map = fdr_dict.get(fdr, {})
            depths = [
                depth
                for terms in lvl_map.values()
                for (_, _, depth) in terms
                if isinstance(depth, int)
            ]
            depth_counters[fdr] = Counter(depths)

        all_depths = sorted({d for cnt in depth_counters.values() for d in cnt})
        if not all_depths:
            print(f"Skipping {species}_{asp}, no depth data")
            continue

        df = pd.DataFrame(
            {f"{fdr:.2f}": [depth_counters[fdr].get(d, 0) for d in all_depths]
             for fdr in target_fdrs},
            index=all_depths
        )
        df.index.name = "Depth"

        fname = f"{species}_{asp}_depth_counts.csv"
        df.to_csv(out_dir / fname)
        print(f"Wrote {fname}")


Wrote PlasmoDB-68_PadleriG01_BP_depth_counts.csv
Wrote PlasmoDB-68_PadleriG01_CC_depth_counts.csv
Wrote PlasmoDB-68_PadleriG01_MF_depth_counts.csv
Wrote PlasmoDB-68_PbergheiANKA_BP_depth_counts.csv
Wrote PlasmoDB-68_PbergheiANKA_CC_depth_counts.csv
Wrote PlasmoDB-68_PbergheiANKA_MF_depth_counts.csv
Wrote PlasmoDB-68_PblacklockiG01_BP_depth_counts.csv
Wrote PlasmoDB-68_PblacklockiG01_CC_depth_counts.csv
Wrote PlasmoDB-68_PblacklockiG01_MF_depth_counts.csv
Wrote PlasmoDB-68_Pchabaudichabaudi_BP_depth_counts.csv
Wrote PlasmoDB-68_Pchabaudichabaudi_CC_depth_counts.csv
Wrote PlasmoDB-68_Pchabaudichabaudi_MF_depth_counts.csv
Wrote PlasmoDB-68_PcoatneyiHackeri_BP_depth_counts.csv
Wrote PlasmoDB-68_PcoatneyiHackeri_CC_depth_counts.csv
Wrote PlasmoDB-68_PcoatneyiHackeri_MF_depth_counts.csv
Wrote PlasmoDB-68_PcynomolgiM_BP_depth_counts.csv
Wrote PlasmoDB-68_PcynomolgiM_CC_depth_counts.csv
Wrote PlasmoDB-68_PcynomolgiM_MF_depth_counts.csv
Wrote PlasmoDB-68_Pfalciparum3D7_BP_depth_counts.csv
Wrote

In [55]:
#save the deepest_annotations dict to a pickle file
with open("genomes_to_annotate_with_PlasmoFP/deepest_annotations.pkl", "wb") as f:
    pickle.dump(deepest_annotations, f)
#save the filtered_deepest dict to a pickle file
with open("genomes_to_annotate_with_PlasmoFP/filtered_deepest.pkl", "wb") as f:
    pickle.dump(filtered_deepest, f)

#load the filtered_deepest dict from the pickle file

In [56]:
# deepest_annotations[species][asp][fdr] = { protein: {(go_id, score, name), …}, … }
import obonet

# 1) load the DO-NOT-ANNOTATE terms
graph = obonet.read_obo("gocheck_do_not_annotate.obo")
DO_NOT_ANNOTATE = set(graph.nodes)

# 2) build a filtered version
filtered_deepest = {}

for species, asp_dict in deepest_annotations.items():
    filtered_deepest[species] = {}
    for asp, fdr_dict in asp_dict.items():
        filtered_deepest[species][asp] = {}
        for fdr, prot_dict in fdr_dict.items():
            new_prot_dict = {}
            for prot, termset in prot_dict.items():
                # drop any tuple whose go_id is in DO_NOT_ANNOTATE
                filtered_terms = {
                    (gid, score, name)
                    for (gid, score, name) in termset
                    if gid not in DO_NOT_ANNOTATE
                }
                if filtered_terms:
                    new_prot_dict[prot] = filtered_terms
            # store only if there's anything left
            filtered_deepest[species][asp][fdr] = new_prot_dict

# Now `filtered_deepest` mirrors `deepest_annotations` but with DO_NOT_ANNOTATE terms removed.


In [57]:
# define the three GO root IDs
ROOT_TERMS = {
    "MF": "GO:0003674",  # molecular_function root
    "BP": "GO:0008150",  # biological_process root
    "CC": "GO:0005575",  # cellular_component root
}

zeroed = {}

for species, asp_dict in filtered_deepest.items():
    zeroed[species] = {}
    for asp, fdr_dict in asp_dict.items():
        root_id = ROOT_TERMS[asp]
        zeroed[species][asp] = {}

        for fdr, prot_dict in fdr_dict.items():
            new_prot_dict = {}
            for prot, termset in prot_dict.items():
                # remove the root term if present
                filtered_terms = {
                    (gid, score, name)
                    for (gid, score, name) in termset
                    if gid != root_id
                }
                # assign even if empty
                new_prot_dict[prot] = filtered_terms

            zeroed[species][asp][fdr] = new_prot_dict

# Now `zeroed` has the exact same structure as `filtered_deepest`,
# except any protein whose only annotation was the root now has an empty set.


In [58]:
import pickle
from pathlib import Path

orig_dir   = Path("genomes_to_annotate_with_PlasmoFP/gene_dicts_out_complete_and_filtered_2")
out_dir    = Path("genomes_to_annotate_with_PlasmoFP/with_PFP_predictions_complete_2")
out_dir.mkdir(exist_ok=True)

pfp_preds = zeroed  # or deepest_annotations

for gene_fp in orig_dir.glob("PlasmoDB-68_*_gene_dict_complete.pkl"):
    species  = gene_fp.stem.rsplit("_gene_dict", 1)[0]
    orig     = pickle.load(open(gene_fp, "rb"))
    augmented = {}

    # 1) pre‐compute which proteins got predictions in each aspect
    proteins_with_preds = {"MF": set(), "BP": set(), "CC": set()}
    for asp, asp_dict in pfp_preds.get(species, {}).items():
        for prot_dict in asp_dict.values():
            proteins_with_preds[asp].update(prot_dict.keys())

    # 2) union across aspects → proteins with any PFP entry
    proteins_with_any = set().union(*proteins_with_preds.values())

    # 3) augment each record, only injecting PFP fields for those in proteins_with_any
    for prot, rec in orig.items():
        new_rec = rec.copy()
        if prot in proteins_with_any:
            for asp, label in [("MF","PFP MF"), ("BP","PFP BP"), ("CC","PFP CC")]:
                aspect_preds = {}
                for fdr, prot_dict in pfp_preds.get(species, {})\
                                                   .get(asp, {})\
                                                   .items():
                    aspect_preds[fdr] = prot_dict.get(prot, set())
                new_rec[label] = aspect_preds
        augmented[prot] = new_rec

    # 4) save out
    out_fp = out_dir / f"{species}_gene_dict_with_PFP.pkl"
    with open(out_fp, "wb") as f:
        pickle.dump(augmented, f)

    # 5) compute and print stats
    total        = len(orig)
    mf_cnt       = len(proteins_with_preds["MF"])
    bp_cnt       = len(proteins_with_preds["BP"])
    cc_cnt       = len(proteins_with_preds["CC"])
    any_cnt      = len(proteins_with_any)
    none_cnt     = total - any_cnt

    print(
        f"{species}: {total} total proteins  →  "
        f"PFP MF: {mf_cnt}, BP: {bp_cnt}, CC: {cc_cnt};  "
        f"{any_cnt} proteins with ≥1 PFP entry, {none_cnt} without"
    )


PlasmoDB-68_PknowlesiH: 5328 total proteins  →  PFP MF: 2220, BP: 2592, CC: 1950;  3261 proteins with ≥1 PFP entry, 2067 without
PlasmoDB-68_PadleriG01: 5325 total proteins  →  PFP MF: 2422, BP: 2863, CC: 2453;  3609 proteins with ≥1 PFP entry, 1716 without
PlasmoDB-68_PvinckeibrucechwattiDA: 5225 total proteins  →  PFP MF: 2259, BP: 2680, CC: 2526;  3456 proteins with ≥1 PFP entry, 1769 without
PlasmoDB-68_PbergheiANKA: 4958 total proteins  →  PFP MF: 1992, BP: 2272, CC: 1712;  2908 proteins with ≥1 PFP entry, 2050 without
PlasmoDB-68_Pgallinaceum8A: 5286 total proteins  →  PFP MF: 2172, BP: 2607, CC: 2073;  3297 proteins with ≥1 PFP entry, 1989 without
PlasmoDB-68_PcoatneyiHackeri: 5516 total proteins  →  PFP MF: 2835, BP: 3247, CC: 2845;  4013 proteins with ≥1 PFP entry, 1503 without
PlasmoDB-68_PmalariaeUG01: 5942 total proteins  →  PFP MF: 2478, BP: 2868, CC: 2035;  3511 proteins with ≥1 PFP entry, 2431 without
PlasmoDB-68_Pyoeliiyoelii17XNL2023: 7047 total proteins  →  PFP MF: 21

In [59]:
import pickle
import pandas as pd
from pathlib import Path

# CONFIG
dict_dir  = Path("genomes_to_annotate_with_PlasmoFP/with_PFP_predictions_complete_2")
out_dir   = Path("genomes_to_annotate_with_PlasmoFP/gene_dicts_out_complete_and_filtered_2/annotation_summaries_2")
out_dir.mkdir(exist_ok=True)

FDRS     = [0.01, 0.05, 0.10, 0.20, 0.30]
GO_KEYS  = [
    ("GO Function", "GO IEA Function"),
    ("GO Process",  "GO IEA Process"),
    ("GO Component","GO IEA Component"),
]
PFP_KEYS = {"MF":"PFP MF","BP":"PFP BP","CC":"PFP CC"}

def triage(triples):
    nonempty = sum(1 for x in triples if len(x) > 0)
    return ("existing" if nonempty == 3
            else "none"    if nonempty == 0
            else "partial")

for pkl_fp in sorted(dict_dir.glob("PlasmoDB-68_*_gene_dict_with_PFP.pkl")):
    species   = pkl_fp.stem.rsplit("_gene_dict",1)[0]
    gene_dict = pickle.load(open(pkl_fp, "rb"))

    # initialize counters
    summary = {
        "Species":       species,
        "Total_genes":   len(gene_dict),
        "GO_existing":   0,
        "GO_partial":    0,
        "GO_none":       0,
    }
    for f in FDRS:
        summary[f"PFP_{f:.2f}_existing"] = 0
        summary[f"PFP_{f:.2f}_partial"]  = 0
        summary[f"PFP_{f:.2f}_none"]     = 0

    # iterate **only** over keys in gene_dict
    for prot, rec in gene_dict.items():
        # A) original GO coverage
        go_sets = [ set(rec[a])|set(rec[b]) for (a,b) in GO_KEYS ]
        cat_go  = triage(go_sets)
        summary[f"GO_{cat_go}"] += 1

        # B) augmented PFP coverage, at each FDR
        for f in FDRS:
            comb = []
            for (a,b), asp in zip(GO_KEYS, ["MF","BP","CC"]):
                orig = set(rec[a])|set(rec[b])
                pfp  = rec.get(PFP_KEYS[asp], {}).get(f, set())
                comb.append(orig | pfp)
            cat_pfp = triage(comb)
            summary[f"PFP_{f:.2f}_{cat_pfp}"] += 1

    # write out one‐row CSV
    df     = pd.DataFrame([summary])
    out_fp = out_dir / f"{species}_annotation_summary.csv"
    df.to_csv(out_fp, index=False)
    print(f"Wrote {out_fp.name}")


Wrote PlasmoDB-68_PadleriG01_annotation_summary.csv
Wrote PlasmoDB-68_PbergheiANKA_annotation_summary.csv
Wrote PlasmoDB-68_PblacklockiG01_annotation_summary.csv
Wrote PlasmoDB-68_Pchabaudichabaudi_annotation_summary.csv
Wrote PlasmoDB-68_PcoatneyiHackeri_annotation_summary.csv
Wrote PlasmoDB-68_PcynomolgiM_annotation_summary.csv
Wrote PlasmoDB-68_Pfalciparum3D7_annotation_summary.csv
Wrote PlasmoDB-68_PfragileNilgiri_annotation_summary.csv
Wrote PlasmoDB-68_PgaboniG01_annotation_summary.csv
Wrote PlasmoDB-68_Pgallinaceum8A_annotation_summary.csv
Wrote PlasmoDB-68_PinuiSanAntonio1_annotation_summary.csv
Wrote PlasmoDB-68_PknowlesiH_annotation_summary.csv
Wrote PlasmoDB-68_PmalariaeUG01_annotation_summary.csv
Wrote PlasmoDB-68_PovalecurtisiGH01_annotation_summary.csv
Wrote PlasmoDB-68_PovalewallikeriPowCR01_annotation_summary.csv
Wrote PlasmoDB-68_PreichenowiCDC_annotation_summary.csv
Wrote PlasmoDB-68_PvinckeibrucechwattiDA_annotation_summary.csv
Wrote PlasmoDB-68_PvivaxSal1_annotation

In [60]:
import pandas as pd
from pathlib import Path

# Path to your per‐species summaries
summaries_dir = Path("genomes_to_annotate_with_PlasmoFP/gene_dicts_out_complete_and_filtered_2/annotation_summaries_2")

# Glob all CSVs
csv_files = sorted(summaries_dir.glob("*_annotation_summary.csv"))

# Read and concatenate
dfs = []
for fp in csv_files:
    df = pd.read_csv(fp)
    dfs.append(df)
master_df = pd.concat(dfs, ignore_index=True)

# Optional: sort by Species
master_df = master_df.sort_values("Species").reset_index(drop=True)
#drop duplicate rows 
master_df = master_df.drop_duplicates(subset=["Species"])
# Save master CSV
out_fp = summaries_dir / "master_annotation_summary.csv"
master_df.to_csv(out_fp, index=False)

print(f"Wrote master summary with {len(master_df)} species to {out_fp}")
print(master_df)


Wrote master summary with 19 species to genomes_to_annotate_with_PlasmoFP/gene_dicts_out_complete_and_filtered_2/annotation_summaries_2/master_annotation_summary.csv
                               Species  Total_genes  GO_existing  GO_partial  \
0               PlasmoDB-68_PadleriG01         5325          872        2748   
2             PlasmoDB-68_PbergheiANKA         4958         1201        2487   
4           PlasmoDB-68_PblacklockiG01         5100          828        2679   
6        PlasmoDB-68_Pchabaudichabaudi         5199         1076        2646   
8         PlasmoDB-68_PcoatneyiHackeri         5516          726        2709   
10             PlasmoDB-68_PcynomolgiM         6068          439        1999   
12          PlasmoDB-68_Pfalciparum3D7         5389         2278        2265   
14         PlasmoDB-68_PfragileNilgiri         5672          732        2722   
16              PlasmoDB-68_PgaboniG01         5134          847        2561   
18          PlasmoDB-68_Pgallinace